## Train

In [ ]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]
!pip install rouge_score
!pip install accelerate -U

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import json

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
model.to('cuda')

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=128, truncation=True, padding=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=128, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [ ]:
json_file_path = "/content/normalized_train_val_test_dataset_2.json"

In [ ]:
input_lines = []
label_lines = []

def load_json_to_dict(file_path):
    with open(file_path, 'r') as f:
        content = json.load(f)
    return content

content = load_json_to_dict(json_file_path)

def get_dataset(content, data_type='train'):
  for sample in content[data_type]:
    input_lines.append(sample["text"] + '</s>')
    label_lines.append(sample["label"])

  dict_obj = {'inputs': input_lines, 'labels': label_lines}
  dataset = Dataset.from_dict(dict_obj)
  tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)
  return dataset, tokenized_datasets

In [ ]:
train_dataset, tokenized_train_datasets = get_dataset(content, 'train')
val_dataset, tokenized_val_datasets = get_dataset(content, 'val')
test_dataset, tokenized_test_datasets = get_dataset(content, 'test')

In [ ]:

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments("/content/results/",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=30,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      evaluation_strategy="epoch",
                                      fp16=True,
                                      )



# AdaFactor for ViT5-large models as it based on T5v1.1.
# See https://medium.com/the-artificial-impostor/paper-adafactor-adaptive-learning-rates-with-sublinear-memory-cost-a543abffa37
#
# from transformers.optimization import Adafactor, AdafactorSchedule
# optimizer = Adafactor(
#     model.parameters(),
#     lr=1e-3,
#     eps=(1e-30, 1e-3),
#     clip_threshold=1.0,
#     decay_rate=-0.8,
#     beta1=None,
#     weight_decay=0.0,
#     relative_step=False,
#     scale_parameter=False,
#     warmup_init=False
# )
# lr_scheduler = AdafactorSchedule(optimizer)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset= tokenized_val_datasets,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.232437
2,No log,0.501309
3,4.064300,0.326283
4,4.064300,0.215991
5,4.064300,0.166128
6,0.398100,0.117687
7,0.398100,0.093830
8,0.215000,0.073551
9,0.215000,0.065252
10,0.215000,0.056522


TrainOutput(global_step=5880, training_loss=0.43347855668489627, metrics={'train_runtime': 1912.4684, 'train_samples_per_second': 12.283, 'train_steps_per_second': 3.075, 'total_flos': 941846148679680.0, 'train_loss': 0.43347855668489627, 'epoch': 30.0})

## Evaluation

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

<ipython-input-13-ffe5587f2e49>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/results/checkpoint-5488")
model.to('cuda')

In [ ]:
import torch
import numpy as np
metrics = load_metric('rouge')

max_target_length = 256
dataloader = torch.utils.data.DataLoader(tokenized_test_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)

metrics.compute()

  0%|          | 0/29 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.9778395630443119, recall=0.9764709530775012, fmeasure=0.9754667209125588), mid=Score(precision=0.9830410955843725, recall=0.9815410848763784, fmeasure=0.9802588440680129), high=Score(precision=0.987763198796047, recall=0.9866135927601867, fmeasure=0.9850410536531697)),
 'rouge2': AggregateScore(low=Score(precision=0.9475181662076251, recall=0.9467116956210816, fmeasure=0.9449757231914465), mid=Score(precision=0.95721004532402, recall=0.9568767202090198, fmeasure=0.954940183685411), high=Score(precision=0.9661960060667094, recall=0.9655882392503778, fmeasure=0.9636712584329362)),
 'rougeL': AggregateScore(low=Score(precision=0.9648847109502823, recall=0.963623299059482, fmeasure=0.9625539630881536), mid=Score(precision=0.9714899530355359, recall=0.9706146487257394, fmeasure=0.9690991254371703), high=Score(precision=0.977641330638732, recall=0.9771714264292248, fmeasure=0.9752958521187943)),
 'rougeLsum': AggregateScore(low=Score(precision=

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

[{'rouge1': 0.9753578292151317},
 {'rouge2': 0.9414661406960341},
 {'rougeL': 0.960186499064932},
 {'rougeLsum': 0.9603734827380485}]

# Inference

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("Thuong/vsl_baseline")
model.to('cuda')

In [ ]:
sentence = "Mưa lũ sạt lở đường ở vùng núi phía Bắc."
text =  sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    early_stopping=True
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

mưa lũ sạt lở đường ở vùng núi phía Bắc


## Upload to Huggingface Hub

In [ ]:
from huggingface_hub import HfApi

from huggingface_hub import notebook_login
notebook_login()

api = HfApi()
username = api.whoami()['name']

try:
  api.create_repo(repo_id="Thuong/vsl_baseline_2")
  print("Successfully created repository !")
except Exception as e:
  print("Repository is possibly already created. Refer to error here - \n\n", e)

api.upload_folder(
    folder_path="/content/results/checkpoint-5880",
    repo_id="Thuong/vsl_baseline_2",
)